# Biomarkers for palbociclib - Workbook 2 June 2023 

## Import Data

In [1]:
import pandas as pd
import pickle

# import GDSC2 drug response data using pickle

with open('data/drug-response/GDSC2/cache_gdsc2.pkl', 'rb') as f:
    gdsc2 = pickle.load(f)
    gdsc2_info = pickle.load(f)

# import CCLE gene expression data using pickle

with open('data/gene-expression/CCLE_Public_22Q2/ccle_expression.pkl', 'rb') as f:
    gene_entrez = pickle.load(f)
    ccle = pickle.load(f)

# import CCLE sample info data using pickle

with open('data/gene-expression/CCLE_Public_22Q2/ccle_sample_info.pkl', 'rb') as f:
    ccle_sample_info = pickle.load(f)

# import STRING database using pickle

with open('data/protein-interaction/STRING/string_df.pkl', 'rb') as f:
    string_df = pickle.load(f)
    string_df_info = pickle.load(f)
    string_df_alias = pickle.load(f)


# import proteomic expression
with open('data/proteomic-expression/goncalves-2022-cell/goncalve_proteome_fillna_processed.pkl', 'rb') as f:
    joined_full_protein_matrix = pickle.load(f)
    joined_sin_peptile_exclusion_matrix = pickle.load(f)

# import STRING database using pickle

with open('data/protein-interaction/STRING/string_df.pkl', 'rb') as f:
    string_df = pickle.load(f)
    string_df_info = pickle.load(f)
    string_df_alias = pickle.load(f)

# open STRING to goncalves mapping file

with open('data\protein-interaction\STRING\goncalve_to_string_id_df.pkl', 'rb') as f:
    goncalve_to_string_id_df = pickle.load(f)


## Palbociclib GDSC with Goncalves et al proteomics (preprocessed & normalised)

In [2]:
# create feature and target 

import DataFunctions as utils

drug_selected = 'Palbociclib'

# create the full dataset

palbociclib_proteomic_df = utils.create_joint_dataset_from_proteome_gdsc(drug_selected, joined_sin_peptile_exclusion_matrix, gdsc2)

feature_data, label_data = utils.create_feature_and_label(palbociclib_proteomic_df)



### Computing Interactors

In [3]:
# using STRING database to select the 1st,2nd and 3rd degree neighbours of the drug target

import pandas as pd

drug_targets = ['CDK4', 'CDK6']
first_degree_neighbours = []
second_degree_neighbours = []
third_degree_neighbours = []

for drug_target in drug_targets:
    string_id = utils.get_protein_id_by_name(drug_target, string_df_info, string_df_alias)
    if string_id is not None:
        first_interactors_string_id = utils.get_protein_interactors(string_id, string_df, score_threshold=900)
        for ii in first_interactors_string_id:
            interactor_name = utils.get_protein_name_by_id(ii, goncalve_to_string_id_df, 
                                                           field_name='goncalve_protein_id',
                                                           check_field_name='string_protein_id')
            if interactor_name is not None:
                first_degree_neighbours.append(interactor_name)

first_degree_neighbours = list(set(first_degree_neighbours))

print(f'first degree neighbours size: {len(first_degree_neighbours)}')
print(f'first degree neighbours: {first_degree_neighbours}')

first degree neighbours size: 43
first degree neighbours: ['P49715;CEBPA_HUMAN', 'P06493;CDK1_HUMAN', 'Q16543;CDC37_HUMAN', 'P49918;CDN1C_HUMAN', 'Q9P2W1;HOP2_HUMAN', 'P08238;HS90B_HUMAN', 'P84022;SMAD3_HUMAN', 'Q13951;PEBB_HUMAN', 'P50613;CDK7_HUMAN', 'P06400;RB_HUMAN', 'Q00535;CDK5_HUMAN', 'Q13485;SMAD4_HUMAN', 'P04637;P53_HUMAN', 'Q00534;CDK6_HUMAN', 'O95067;CCNB2_HUMAN', 'O43502;RA51C_HUMAN', 'P20248;CCNA2_HUMAN', 'P12004;PCNA_HUMAN', 'P11802;CDK4_HUMAN', 'P07948;LYN_HUMAN', 'Q13547;HDAC1_HUMAN', 'P07900;HS90A_HUMAN', 'P50750;CDK9_HUMAN', 'P00519;ABL1_HUMAN', 'P49841;GSK3B_HUMAN', 'O60563;CCNT1_HUMAN', 'Q13309;SKP2_HUMAN', 'P31947;1433S_HUMAN', 'P51948;MAT1_HUMAN', 'P42773;CDN2C_HUMAN', 'O75832;PSD10_HUMAN', 'P12931;SRC_HUMAN', 'P14635;CCNB1_HUMAN', 'P24385;CCND1_HUMAN', 'Q9BWT6;MND1_HUMAN', 'P51946;CCNH_HUMAN', 'P15090;FABP4_HUMAN', 'P16989;YBOX3_HUMAN', 'P24941;CDK2_HUMAN', 'P10275;ANDR_HUMAN', 'P42771;CDN2A_HUMAN', 'Q9P287;BCCIP_HUMAN', 'Q14186;TFDP1_HUMAN']


In [4]:
# get the second degree neighbours using first_interactors_string_id

for ii in first_interactors_string_id:
    second_interactors_string_id = utils.get_protein_interactors(ii, string_df, score_threshold=900)
    for sec_ii in second_interactors_string_id:
        interactor_name = utils.get_protein_name_by_id(sec_ii, goncalve_to_string_id_df, 
                                                       field_name='goncalve_protein_id',
                                                       check_field_name='string_protein_id')
        if interactor_name is not None:
            second_degree_neighbours.append(interactor_name)

second_degree_neighbours = list(set(second_degree_neighbours + first_degree_neighbours))
print(f'second degree neighbours size: {len(second_degree_neighbours)}')
print(f'second degree neighbours: {second_degree_neighbours}')



second degree neighbours size: 967
second degree neighbours: ['P09874;PARP1_HUMAN', 'Q9NTJ3;SMC4_HUMAN', 'P49918;CDN1C_HUMAN', 'O15164;TIF1A_HUMAN', 'Q9BX66;SRBS1_HUMAN', 'P41226;UBA7_HUMAN', 'P08238;HS90B_HUMAN', 'Q9UBU8;MO4L1_HUMAN', 'Q9Y6A5;TACC3_HUMAN', 'P12830;CADH1_HUMAN', 'P51608;MECP2_HUMAN', 'O00422;SAP18_HUMAN', 'Q15554;TERF2_HUMAN', 'Q01201;RELB_HUMAN', 'P08138;TNR16_HUMAN', 'P11532;DMD_HUMAN', 'P62826;RAN_HUMAN', 'Q9UIG0;BAZ1B_HUMAN', 'Q00535;CDK5_HUMAN', 'P52294;IMA5_HUMAN', 'Q13671;RIN1_HUMAN', 'P31948;STIP1_HUMAN', 'P20936;RASA1_HUMAN', 'Q16763;UBE2S_HUMAN', 'Q9BQG0;MBB1A_HUMAN', 'Q9UJU2;LEF1_HUMAN', 'O14558;HSPB6_HUMAN', 'Q14195;DPYL3_HUMAN', 'P28074;PSB5_HUMAN', 'P30273;FCERG_HUMAN', 'P83916;CBX1_HUMAN', 'Q9NP50;SHCAF_HUMAN', 'P35609;ACTN2_HUMAN', 'Q08722;CD47_HUMAN', 'Q96ST3;SIN3A_HUMAN', 'P61956;SUMO2_HUMAN', 'P04632;CPNS1_HUMAN', 'Q14247;SRC8_HUMAN', 'P09471;GNAO_HUMAN', 'P16220;CREB1_HUMAN', 'Q15528;MED22_HUMAN', 'Q13469;NFAC2_HUMAN', 'Q13555;KCC2G_HUMAN', 'Q8IX90;

In [5]:
# get the third degree neighbours using second_interactors_string_id

for ii in second_interactors_string_id:
    third_interactors_string_id = utils.get_protein_interactors(ii, string_df, score_threshold=900)
    for third_ii in third_interactors_string_id:
        interactor_name = utils.get_protein_name_by_id(third_ii, goncalve_to_string_id_df, 
                                                       field_name='goncalve_protein_id',
                                                       check_field_name='string_protein_id')
        if interactor_name is not None:
            third_degree_neighbours.append(interactor_name)

third_degree_neighbours = list(set(third_degree_neighbours + second_degree_neighbours))
print(f'third degree neighbours size: {len(third_degree_neighbours)}')
print(f'third degree neighbours: {third_degree_neighbours}')

third degree neighbours size: 993
third degree neighbours: ['P09874;PARP1_HUMAN', 'Q9NTJ3;SMC4_HUMAN', 'P49918;CDN1C_HUMAN', 'Q9UBU8;MO4L1_HUMAN', 'P51608;MECP2_HUMAN', 'P41226;UBA7_HUMAN', 'P08238;HS90B_HUMAN', 'O15164;TIF1A_HUMAN', 'Q9Y6A5;TACC3_HUMAN', 'P12830;CADH1_HUMAN', 'O00422;SAP18_HUMAN', 'Q9UH99;SUN2_HUMAN', 'Q9BX66;SRBS1_HUMAN', 'Q15554;TERF2_HUMAN', 'Q01201;RELB_HUMAN', 'P08138;TNR16_HUMAN', 'P11532;DMD_HUMAN', 'P62826;RAN_HUMAN', 'Q9UIG0;BAZ1B_HUMAN', 'Q00535;CDK5_HUMAN', 'P52294;IMA5_HUMAN', 'Q13671;RIN1_HUMAN', 'P31948;STIP1_HUMAN', 'P20936;RASA1_HUMAN', 'Q16763;UBE2S_HUMAN', 'Q9BQG0;MBB1A_HUMAN', 'Q9UJU2;LEF1_HUMAN', 'O14558;HSPB6_HUMAN', 'P28074;PSB5_HUMAN', 'P30273;FCERG_HUMAN', 'Q14195;DPYL3_HUMAN', 'Q9NP50;SHCAF_HUMAN', 'P83916;CBX1_HUMAN', 'P35609;ACTN2_HUMAN', 'Q08722;CD47_HUMAN', 'P01137;TGFB1_HUMAN', 'Q96ST3;SIN3A_HUMAN', 'P61956;SUMO2_HUMAN', 'P04632;CPNS1_HUMAN', 'Q14247;SRC8_HUMAN', 'P09471;GNAO_HUMAN', 'Q00978;IRF9_HUMAN', 'P16220;CREB1_HUMAN', 'Q15528;MED2

In [6]:
# verify a list is unique

def verify_unique_list(l):
    return len(l) == len(set(l))

# find duplicates in the list

def find_duplicates(l):
    return list(set([x for x in l if l.count(x) > 1]))

print(f'first degree neighbours is unique: {verify_unique_list(first_degree_neighbours)}')
print(f'second degree neighbours is unique: {verify_unique_list(second_degree_neighbours)}')
print(f'third degree neighbours is unique: {verify_unique_list(third_degree_neighbours)}')

# print the duplicates in first degree neighbours

print(f'duplicates in first degree neighbours: {find_duplicates(first_degree_neighbours)}')
print(f'duplicates in second degree neighbours: {find_duplicates(second_degree_neighbours)}')
print(f'duplicates in third degree neighbours: {find_duplicates(third_degree_neighbours)}')

first degree neighbours is unique: True
second degree neighbours is unique: True
third degree neighbours is unique: True
duplicates in first degree neighbours: []
duplicates in second degree neighbours: []
duplicates in third degree neighbours: []


### Validation Framework Implementation

#### Initial Parameters

In [11]:
import Visualisation as vis
import matplotlib.pyplot as plt

import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# import random forest regression model
from sklearn.ensemble import RandomForestRegressor

# import support vector machine regression model
from sklearn.svm import SVR

# import elastic net regression model
from sklearn.linear_model import ElasticNet

# import simple mlp regression model
from sklearn.neural_network import MLPRegressor

# import xgb regression model
from xgboost import XGBRegressor

# import k nearest neighbors regression model
from sklearn.neighbors import KNeighborsRegressor

## feature selection
# import feature selection
from sklearn.feature_selection import SelectKBest, f_regression
import shap 

## validation
from sklearn.metrics import r2_score
from scipy.stats import pearsonr

## saving and loading files
import pickle


## INPUTS 
# file names 
folder_path = 'data/processed-results/workbook-jun-2023/'
input_parameter_file_path = folder_path + 'input_parameters_test.pkl'
output_file_path = folder_path + 'results_test.pkl'
experiment_name = 'test'
save_input = True
save_output = True

# hyperparameters
max_gene_target_disance = 2 # specify the level of biological relevance to drug target(s)
statistical_filter_size = 100 # can be optimized using global feature dropout testing
monte_carlo_cross_validation_size = 3 # can be automatically optimized via rank impact assessment
models_used = ['ElasticNet', 'RandomForestRegressor']
models_hyperparameters = [{}, {}]

# extra hyperparameters
statistical_filter_threshold = 0.05 # currently not in use
cv_split_size = 0.1

# generated hyperparameters
rng_seed_lists = []
for i in range(monte_carlo_cross_validation_size):
    rng_seed_lists.append(np.random.randint(100000))

def get_model_from_string(model_name, **kwargs):
    if model_name == 'ElasticNet':
        return ElasticNet(**kwargs)
    elif model_name == 'RandomForestRegressor':
        return RandomForestRegressor(**kwargs)
    elif model_name == 'SVR':
        return SVR(**kwargs)
    elif model_name == 'MLPRegressor':
        return MLPRegressor(**kwargs)
    elif model_name == 'XGBRegressor':
        return XGBRegressor(**kwargs)
    elif model_name == 'KNeighborsRegressor':
        return KNeighborsRegressor(**kwargs)
    else:
        raise ValueError(f'{model_name} is not supported')
    
nth_degree_neighbours = [drug_targets, first_degree_neighbours, second_degree_neighbours, third_degree_neighbours]

if save_input:
    # save initial parameters as pickle 
    with open(input_parameter_file_path, 'wb') as f:
        # dump each variable individually
        pickle.dump(max_gene_target_disance, f)
        pickle.dump(statistical_filter_size, f)
        pickle.dump(monte_carlo_cross_validation_size, f)
        pickle.dump(models_used, f)
        pickle.dump(models_hyperparameters, f)
        pickle.dump(statistical_filter_threshold, f)
        pickle.dump(cv_split_size, f)
        pickle.dump(input_parameter_file_path, f)
        pickle.dump(output_file_path, f)
        pickle.dump(rng_seed_lists, f)
        pickle.dump(nth_degree_neighbours, f)
        pickle.dump(experiment_name, f)

In [12]:
network_features = nth_degree_neighbours[max_gene_target_disance]
X_train, X_test, y_train, y_test = train_test_split(feature_data, label_data, test_size=cv_split_size,
                                                    random_state=rng_seed_lists[0])

if statistical_filter_size > len(network_features):
    statistical_filter_size = len(network_features)
    print(f'WARNING: statistical_filter_size is too large, set to {statistical_filter_size}')

# perform feature selection on the training set
selector = SelectKBest(f_regression, k=statistical_filter_size)
selector.fit(X_train[network_features], y_train)

# get the selected features
selected_features = X_train[network_features].columns[selector.get_support()]

# get the feature importance
feature_importance = selector.scores_[selector.get_support()]

# DEBUG print the selected features and their importance
# print(f'selected features: {selected_features}')
# print(f'feature importance: {feature_importance}')

In [14]:
X_train.shape

(663, 6692)

#### Feature Selection and Consensus Feature Contribution

In [19]:
verbose = True
max_feature_save_size = 1000
data_collector = []

def get_shap_values(model_str, train_data, test_data):
    if model_str == 'RandomForestRegressor':
        explainer = shap.TreeExplainer(model, train_data)
    elif model_str == 'ElasticNet':
        explainer = shap.LinearExplainer(model, train_data)
    elif model_str == 'XGBRegressor':
        explainer = shap.TreeExplainer(model, train_data)
    elif model_str == 'MLPRegressor':
        explainer = shap.DeepExplainer(model, train_data)
    else:
        explainer = shap.KernelExplainer(model.predict, train_data)
    shap_values = explainer.shap_values(test_data)
    return shap_values

def get_network_stat_features(X_train, y_train, X_test):
    network_features = nth_degree_neighbours[max_gene_target_disance]
            # perform feature selection on the training set
    selector = SelectKBest(f_regression, k=statistical_filter_size)
    selector.fit(X_train[network_features], y_train)
            # get the selected features
    selected_features = X_train[network_features].columns[selector.get_support()]
    sel_train, sel_test = X_train[selected_features], X_test[selected_features]
    return selected_features, sel_train, sel_test

def get_random_features(X_train, y_train, X_test):
    random_features = np.random.choice(X_train.columns, statistical_filter_size, replace=False)
    sel_train, sel_test = X_train[random_features], X_test[random_features]
    return random_features, sel_train, sel_test

def get_all_features(X_train, y_train, X_test):
    sel_train, sel_test = X_train, X_test
    return None, sel_train, sel_test

conditions_to_test = ['network_f_regression_selection', 'whole_dataset_control', 'random_control']
conditions_to_get_feature_importance = [True, False, False]
matched_functions = [get_network_stat_features, get_random_features, get_all_features]

for model_str in models_used:
    for rng in rng_seed_lists:
        X_train, X_test, y_train, y_test = train_test_split(feature_data, label_data, test_size=cv_split_size,
                                                            random_state=rng)

        for i, condition in enumerate(conditions_to_test):
            if verbose:
                print(f'running {model_str} with seed {rng} under {condition} conditions')
            selected_features, sel_train, sel_test = matched_functions[i](X_train, y_train, X_test)
            if verbose:
                if selected_features is not None:
                    print(f'--- selected features: {selected_features.shape}, train data: {sel_train.shape}, test data: {sel_test.shape}')
            model = get_model_from_string(model_str, **models_hyperparameters[models_used.index(model_str)])
            model.fit(sel_train, y_train)
            y_pred = model.predict(sel_test)
            score = mean_squared_error(y_test, y_pred)
            corr, p_val = pearsonr(y_test, y_pred)
            r_squared = r2_score(y_test, y_pred)

            shap_values = None        
            if conditions_to_get_feature_importance[i]:
                shap_values = get_shap_values(model_str, sel_train, sel_test)

            if verbose:
                print(f'--- result: prediction correlation {corr}, p-value {p_val}, r-squared {r_squared}')

            # if sel_train and sel_test are too big, they will not be saved
            if sel_train.shape[1] > max_feature_save_size:
                sel_train = None
            if sel_test.shape[1] > max_feature_save_size:
                sel_test = None

            data_collector.append([rng, model_str, condition, selected_features, 
                                   score, corr, p_val, r_squared, shap_values, 
                                   sel_train, sel_test, y_test, y_pred])
            
if verbose:
    print('### All models ran')

df = pd.DataFrame(data_collector, columns=['rng', 'model', 'exp_condition', 'selected_features'
                                           'mse', 'corr', 'p_val', 'r_squared', 'shap_values', 
                                           'X_train', 'X_test', 'y_test', 'y_pred'])

if save_output:
    with open(output_file_path, 'wb') as f:
        pickle.dump(df, f)

if verbose:
    print('### Results saved')

running ElasticNet with seed 59713 under network_f_regression_selection conditions
--- selected features: (100,), train data: (663, 100), test data: (74, 100)
--- result: prediction correlation 0.32533435563993124, p-value 0.004678622472588647, r-squared 0.05374672167717698
running ElasticNet with seed 59713 under whole_dataset_control conditions
--- selected features: (100,), train data: (663, 100), test data: (74, 100)
--- result: prediction correlation 0.33197783526105323, p-value 0.0038573931651607195, r-squared 0.10269152460687048
running ElasticNet with seed 59713 under random_control conditions
--- result: prediction correlation 0.426670515420615, p-value 0.00015017196076906996, r-squared 0.15675123139039604
running ElasticNet with seed 10772 under network_f_regression_selection conditions
--- selected features: (100,), train data: (663, 100), test data: (74, 100)
--- result: prediction correlation 0.6116497876837123, p-value 7.074970564632639e-09, r-squared 0.30930674285093385


KeyboardInterrupt: 

In [ ]:
df.head()


,rng,model,exp_condition,mse,corr,p_val,r_squared,shap_values,X_train,X_test,y_test,y_pred
0,47049,KNeighborsRegressor,experimental,2.583495,0.418560,2.062653e-04,0.070137,"[[0.0, 0.007731481176660649, 0.0, 0.0, 0.0, 0....",P41240;CSK_HUMAN Q09028;RBBP4_HUMA...,P41240;CSK_HUMAN Q09028;RBBP4_HUMA...,SIDM00351 1.793823 SIDM00444 2.435937 SI...,"[4.057165199999999, 3.2529334, 4.6611974, 4.27..."
1,47049,KNeighborsRegressor,whole_dataset_control,1.684965,0.631445,1.620618e-09,0.393540,None,None,None,SIDM00351 1.793823 SIDM00444 2.435937 SI...,"[3.7660544000000002, 3.8445294000000003, 3.574..."
2,47049,KNeighborsRegressor,random_control,2.345986,0.445643,6.922232e-05,0.155623,None,P61201;CSN2_HUMAN Q9NZK5;ADA2_HUMA...,P61201;CSN2_HUMAN Q9NZK5;ADA2_HUMA...,SIDM00351 1.793823 SIDM00444 2.435937 SI...,"[4.0383336, 2.4861940000000002, 3.455445799999..."
3,15947,KNeighborsRegressor,experimental,2.227222,0.455060,4.632238e-05,0.112793,"[[-0.006201244415312335, -0.013240902228980223...",P41240;CSK_HUMAN Q09028;RBBP4_HUMA...,P41240;CSK_HUMAN Q09028;RBBP4_HUMA...,SIDM01068 5.285555 SIDM00411 4.536486 SI...,"[4.005414, 4.768056199999999, 2.233085, 2.4173..."
4,15947,KNeighborsRegressor,whole_dataset_control,1.972941,0.529664,1.218880e-06,0.214085,None,None,None,SIDM01068 5.285555 SIDM00411 4.536486 SI...,"[5.0939928, 4.6528128, 2.54835, 1.514397600000..."


In [ ]:
def get_mean_contribution(df):
    # df: dataframe with shap_values, X_train and X_test column
    # extract all the shap values, match the feature names and store them in a dataframe

    # for the df, select only the row with the exp_condition column == 'experimental'
    df = df[df['exp_condition'] == 'experimental']

    collector = []
    for shap, x_test in zip(df['shap_values'], df['X_test']):
        # print(shap.shape, cols.shape)
        mean_shap = np.abs(shap).mean(axis=0)
        column_names = x_test.columns
        joint_data = list(zip(column_names, mean_shap))
        # sort the joint data by column names
        joint_data.sort(key=lambda x: x[0])
        collector.append(joint_data)

    # create a dataframe from the collector

    # first, create a list of column names

    column_names = [x[0] for x in collector[0]]

    shap_df = pd.DataFrame(collector, columns=column_names)

    # for every cell in the dataframe, keep only the shap value, which is the second element in the tuple

    for col in shap_df.columns:
        shap_df[col] = shap_df[col].apply(lambda x: x[1])

    # sort the dataframe columns by the mean shap values

    shap_df = shap_df.reindex(shap_df.mean().sort_values(ascending=False).index, axis=1)
    shap_df.head()


    # compute the mean shap values for each column

    mean_shap_values = shap_df.mean()
    return mean_shap_values

#### in silico Knockout Validation 

are the features really important? Knock them out and check if the model performance decreases, compare it with a random feature knockout. If not, then what new features are contributing to the model performance?